In [1]:
!pip install --upgrade  textblob gensim pytorch-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: textblob
    Found existing installation: textblob 0.17.1
    Uninstalling textblob-0.17.1:
      Successfully uninstalled textblob-0.17.1


In [2]:
import multiprocessing
import torch
import torch.nn as nn
import torch.optim as optim
import itertools
import sys
from textblob import TextBlob, Word
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False  # Setting to true is useful when debugging to know which device is being used
embedding_dim = 50
epochs=100
batch_size = 100
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  random.seed(42)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

Writing get_data.sh


In [4]:
!bash get_data.sh

--2024-10-08 07:09:34--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/2k8im8ftu9yk8mnqhops9/yelp.csv?rlkey=52dzmxgys0su77wb6o75vb5ab&dl=0 [following]
--2024-10-08 07:09:35--  https://www.dropbox.com/scl/fi/2k8im8ftu9yk8mnqhops9/yelp.csv?rlkey=52dzmxgys0su77wb6o75vb5ab&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc90645b157a9d890d144911c1e7.dl.dropboxusercontent.com/cd/0/inline/CcDDYoERoZjVELZvDCpl_nTOH5kOEtQJf_hcuI3apS_85h9SeiLDleDZEPJjhmCtU2wXSHTZ0Atyn0B-fF6TFscnZSbdKwpbJww3NGvcUpMKEQA01sCHSY_Q2NNMTcR7dyOqphDSzzUTksFUVuwpzVHX/file# [following]
--2024-10-08 07:09:36--  https://uc90645b157a9d890d144911c1e7.dl.dropboxusercontent.com/cd/0/in

In [5]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [6]:
yelp

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,VY_tvNUCCXGXQeSvJl757Q,2012-07-28,Ubyfp2RSDYW0g7Mbr8N3iA,3,First visit...Had lunch here today - used my G...,review,_eqQoPtQ3e3UxLE4faT6ow,1,2,0
9996,EKzMHI1tip8rC1-ZAy64yg,2012-01-18,2XyIOQKbVFb6uXQdJ0RzlQ,4,Should be called house of deliciousness!\n\nI ...,review,ROru4uk5SaYc3rg8IU7SQw,0,0,0
9997,53YGfwmbW73JhFiemNeyzQ,2010-11-16,jyznYkIbpqVmlsZxSDSypA,4,I recently visited Olive and Ivy for business ...,review,gGbN1aKQHMgfQZkqlsuwzg,0,0,0
9998,9SKdOoDHcFoxK5ZtsgHJoA,2012-12-02,5UKq9WQE1qQbJ0DJbc-B6Q,2,My nephew just moved to Scottsdale recently so...,review,0lyVoNazXa20WzUyZPLaQQ,0,0,0


In [7]:
TextBlob(X.values[0]).correct()

TextBlob("By wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our witness was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

To yourself a favor and get their Bloody Mary.  It was phenomena and simply the best I've ever had.  I'm pretty sure they only use ingredient from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white ruffle scrambled eggs vegetable skilled and it was taste and delicious.  It came with 2 pieces of their grizzled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!")

In [8]:
# Create corpus of sentences such that the sentence has more than 3 words
corpus = [line for line in X.values if len(TextBlob(line).words)> 3]

In [9]:
corpus[:2]

['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!',
 'I have no idea why some people give bad reviews about this place. It goes to show you, you can

At this point we have a list (any iterable will do) of queries that are longer than 3 words. This is normal to filter random queries. Now we must use the `LabelEncoder` object to `fit` on the corpus, in order to convert each wor to an ID, and later convert such corpus of list of words into their identifiers.


In [12]:
!pip install --upgrade swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 24.3 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=d5cfd8d9657d83450e9e0100ec5248e34a80b66d3a2ea4facc65db36a0ae4a30
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [23]:
#tokenizer = Tokenizer()
# # Use the fit_on_texts method to fit the tokenizer
#tokenizer.fit_on_texts(corpus) # Fill

import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in textblob_tokenizer(text))
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    text_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords
    text_token_list = [word_rooter(word) if '#' not in word else word
                        for word in text_token_list] # apply word rooter
    text = ' '.join(text_token_list)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

import swifter
X_preprocessed = X.swifter.apply(preprocess_text)

def get_maximum_review_length(srs):
    maximum = 0
    for response in srs:
        candidate = len(preprocess_text(response, should_join=False))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(X_preprocessed)

import itertools
from torchnlp.encoders import LabelEncoder
list_of_words = list(itertools.chain.from_iterable([preprocess_text(sentence, should_join=False) for sentence in X_preprocessed.values]))
ids_from_words = LabelEncoder(list_of_words, reserved_labels=['UNK'], unknown_index=0)

print(f'Before the tokenizer: {corpus[:1]}')

#Now use the same "trained" tokenizer to convert the corpus from words to IDs with the texts_to_sequences method
tokenized_corpus = [ids_from_words.batch_encode(sentence.split()) for sentence in corpus]

print(f'After the tokenizer: {tokenized_corpus[:1]}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pandas Apply:   0%|          | 0/4086 [00:00<?, ?it/s]

Before the tokenizer: ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!']
After the tokenizer: [tensor([ 0,  1,  2,  0,  0,  0,  0,  3,  0,  4,  0, 

In [24]:
vocab_size = LabelEncoder.vocab_size


In [25]:
print(f'First 5 corpus items are {tokenized_corpus[:5]}')
print(f'Length of corpus is {len(tokenized_corpus)}')



First 5 corpus items are [tensor([ 0,  1,  2,  0,  0,  0,  0,  3,  0,  4,  0,  0,  0,  0,  0,  6,  0,  7,
         0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15,  0,  0,  0,  0, 16,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 24,  0, 25,  0,  0,  0,  0,  0,  0, 28,
         0, 29,  0,  0,  0,  0,  0,  0, 31,  0, 29,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 36,  0, 37,  0,  0,  0, 38,  0,  0, 39,  0,  0,  0, 41,  0, 42,
         0, 43,  0,  0, 44,  0,  0,  0,  0,  0,  0,  0,  0, 47,  0,  0,  0,  0,
         0, 48,  0,  0,  0,  0, 53,  0,  0,  0,  0,  0,  0,  0, 56,  0,  0,  0,
         0,  0,  0, 59,  0,  0,  0,  0,  0,  0,  8,  0, 60,  0,  0,  0,  0, 36,
         0,  0, 37,  0,  0,  0,  0, 65,  0, 66,  0]), tensor([    0,     0, 12978,  3284,     0,     0,     0,    70,    71,     0,
            0,     0,     0,     0,     0,     0,    74,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    

In [16]:
type(tokenized_corpus)

list

In [17]:
def ids_from_text(text):
  return ids_from_words.batch_encode(text)

def text_from_ids(ids):
  return ids_from_words.batch_decode(ids)

In [18]:
def pad_sequence_of_tokens(x, maxlen, unk_token='UNK'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

In [46]:
# This is the algorithmic part of batching the dataset and yielding the window of words and expected middle word for each bacth as a generator.
def create_context_target_pairs(texts, context_size):
    data = []
    for text in texts:
        tokens = text.split()
        for i, word in enumerate(tokens):
            start = max(0, i - context_size)
            end = min(len(tokens), i + context_size + 1)
            context = pad_sequence_of_tokens([tokens[j] for j in range(start, end) if j != i], maxlen=4)
            if word not in ids_from_words.vocab:
              target=0
            else :
              target = ids_from_words.token_to_index[word]
            context_indices=[]
            for w in context:
              if w not in ids_from_words.vocab:
                context_indices.append(0)
              else:
                context_indices.append(ids_from_words.token_to_index[w])
            context_indices.append(target)
            data.append(torch.Tensor(context_indices))
    return data

Notice now in a sample how we construct X and y to predict words

In [47]:
data = create_context_target_pairs(corpus[:5], 2)

In [48]:
data = torch.stack(data)

In [49]:
X = data[:, :4].to(torch.long)
y = data[:, 4].to(torch.long)

In [50]:
X,y

(tensor([[  1,   2,   0,   0],
         [  0,   2,   0,   0],
         [  0,   1,   0,   0],
         ...,
         [  0,   0,   0,   0],
         [  0, 137,   0,   0],
         [137,   0,   0,   0]]),
 tensor([    0,     1,     2,     0,     0,     0,     0,     3,     0,     4,
             0,     0,     0,     0,     0,     6,     0,     7,     0,     8,
             0,     0,     0,     0,     0,     0,     0,     0,     0,    15,
             0,     0,     0,     0,    16,     0,     0,     0,     0,     0,
             0,     0,     0,     0,    24,     0,    25,     0,     0,     0,
             0,     0,     0,    28,     0,    29,     0,     0,     0,     0,
             0,     0,    31,     0,    29,     0,     0,     0,     0,     0,
             0,     0,     0,     0,    36,     0,    37,     0,     0,     0,
            38,     0,     0,    39,     0,     0,     0,    41,     0,    42,
             0,    43,     0,     0,    44,     0,     0,     0,     0,     0,
        

In [51]:
print(X.shape)
print(y.shape)


torch.Size([866, 4])
torch.Size([866])


Now comes the core part, defining the model. Keras provides a convenient Sequential model class to just `add` layers of any type and they will just work. Let's add an `Embedding` layer (that will map the word ids into a vector of size 100), a `Lambda` to average the words out in a sentence, and a `Dense layer` to select the best word on the other end. This is classic CBOW.


In [52]:

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        print((vocab_size, embedding_dim))
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # Linear layer to act as the hidden layer
        self.linear1 = nn.Linear(embedding_dim, 128)
        # Linear layer to predict the center word
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = torch.mean(embeds, dim=1)
        out = torch.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = torch.log_softmax(out, dim=1)
        return log_probs

In [53]:
def train_cbow(X, y, model, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        total_loss = 0

        # Step 1. Recall that torch *accumulates* gradients. Before passing in a new instance,
        # you need to zero out the gradients from the old instance
        optimizer.zero_grad()

        # Step 2. Run the forward pass, getting log probabilities over next words
        log_probs = model(X)

        # Step 3. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, y)

        # Step 4. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Print progress
        if (epoch + 1) % 10 == 0:
            print('Epoch: {}, Loss: {:.4f}'.format(epoch + 1, total_loss))
    return model

In [54]:
context_size=2
embedding_dim=50
vocab_size = len(ids_from_words.vocab)
model = CBOW(vocab_size, embedding_dim, context_size * 2)
model.to(device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
X = X.to(device)
y = y.to(device)

(13174, 50)


In [55]:
trained_model = train_cbow(X, y, model, loss_function, optimizer, epochs=100)

Epoch: 10, Loss: 9.5728
Epoch: 20, Loss: 9.3761
Epoch: 30, Loss: 9.1785
Epoch: 40, Loss: 8.9779
Epoch: 50, Loss: 8.7720
Epoch: 60, Loss: 8.5590
Epoch: 70, Loss: 8.3377
Epoch: 80, Loss: 8.1052
Epoch: 90, Loss: 7.8591
Epoch: 100, Loss: 7.5975


In [57]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

embeddings = trained_model.embeddings.weight.data.cpu().numpy()

# Now, we need to save these embeddings in a format that gensim can understand
# For that, we will use the KeyedVectors instance in gensim

# Instantiate the KeyedVectors with the correct size
kv = KeyedVectors(vector_size=embeddings.shape[1])






In [58]:
# Add the vectors and their corresponding words to the KeyedVectors instance
kv.add_vectors(ids_from_words.index_to_token, embeddings)

In [72]:
kv.most_similar(positive=['wife'])

[('gainey', 0.523906409740448),
 ('ristor', 0.5074626207351685),
 ('planet', 0.4803556799888611),
 ('pickup', 0.4775352478027344),
 ('stamina', 0.4760451912879944),
 ('yummmmmmer', 0.46339768171310425),
 ('burch', 0.4613041877746582),
 ('superfood', 0.4558057188987732),
 ('barrel', 0.4547787010669708),
 ('brief', 0.4511219561100006)]

In [71]:
kv.most_similar(negative=['food'])

[('kota', 0.5309324860572815),
 ('erj', 0.493743360042572),
 ('skip', 0.4752471148967743),
 ('taunt', 0.4697993993759155),
 ('aloha', 0.4501955509185791),
 ('hu', 0.4487742483615875),
 ('checklist', 0.44213777780532837),
 ('unemploy', 0.43824273347854614),
 ('hoegaarden', 0.43323206901550293),
 ('zucchini', 0.42678308486938477)]

In [73]:
kv.most_similar(positive=['apple'])

KeyError: "Key 'apple' not present in vocabulary"

In [77]:
kv.vectors.shape

(13174, 50)